# Image based age detection

In [ ]:
#!pip install opencv-contrib-python

In [ ]:
#!pip install function

In [3]:
# import packages
import numpy as np 
import argparse
import cv2
import os
import function

In [2]:
#construct argument parse and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-i", "--image", required=False, help="path to input image")
ap.add_argument("-f", "--face", required=False, help="path to face detector model directory")
ap.add_argument("-a", "--age", required=False, help="path to age detector model directory")
ap.add_argument("-c", "--confidence", type=float, default=0.5, help="minimum probability to filter weak detections")
args = vars(ap.parse_args())

In [4]:
# define the list of age buckets our age detector will predict
AGE_BUCKETS = ["(0-2)", "(4-6)", "(8-12)", "(15-20)", "(25-32)", "(38-43)", "(48-53)", "(60-100)"]

In [5]:
print("[INFO] loading face detector model...")
faceNet = cv2.dnn.readNet("/Users/patsnap/Desktop/Neo4J_and_other_codes/Image_search/OpenCV_age_detection/face_detector/deploy.prototxt", "/Users/patsnap/Desktop/Neo4J_and_other_codes/Image_search/OpenCV_age_detection/face_detector/res10_300x300_ssd_iter_140000.caffemodel")

[INFO] loading face detector model...


In [6]:
print("[INFO] loading age detector model...")
ageNet = cv2.dnn.readNet("/Users/patsnap/Desktop/Neo4J_and_other_codes/Image_search/OpenCV_age_detection/age_detector/age_deploy.prototxt", "/Users/patsnap/Desktop/Neo4J_and_other_codes/Image_search/OpenCV_age_detection/age_detector/age_net.caffemodel")

[INFO] loading age detector model...


In [8]:
import os
filelist=os.listdir('images')
print(filelist)

['samuel_l_jackson.png', 'adrian.png', 'neil_patrick_harris.png']


In [9]:
for image in filelist:
    image = cv2.imread("/Users/patsnap/Desktop/Neo4J_and_other_codes/Image_search/OpenCV_age_detection/images/samuel_l_jackson.png")
    (h, w) = image.shape[:2]
    blob = cv2.dnn.blobFromImage(image, 1.0, (300, 300), (104.0, 177.0, 123.0))
    faceNet.setInput(blob)
    detections = faceNet.forward()

In [ ]:
# loop over the detections
for i in range(0, detections.shape[2]):
    # extract the confidence (i.e., probability) associated with the
    # prediction
    confidence = detections[0, 0, i, 2]

    # filter out weak detections by ensuring the confidence is
    # greater than the minimum confidence
    if confidence > args["confidence"]:
        # compute the (x, y)-coordinates of the bounding box for the
        # object
        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype("int")

        # extract the ROI of the face and then construct a blob from
        # *only* the face ROI
        face = image[startY:endY, startX:endX]
        faceBlob = cv2.dnn.blobFromImage(face, 1.0, (227, 227), (78.4263377603, 87.7689143744, 114.895847746), swapRB=False)

        # make predictions on the age and find the age bucket with
        # the largest corresponding probability
        ageNet.setInput(faceBlob)
        preds = ageNet.forward()
        i = preds[0].argmax()
        age = AGE_BUCKETS[i]
        ageConfidence = preds[0][i]

        # display the predicted age to our terminal
        text = "{}: {:.2f}%".format(age, ageConfidence * 100)
        print("[INFO] {}".format(text))

        # draw the bounding box of the face along with the associated
        # predicted age
        y = startY - 10 if startY - 10 > 10 else startY + 10
        cv2.rectangle(image, (startX, startY), (endX, endY), (0, 0, 255), 2)
        cv2.putText(image, text, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)


In [ ]:
# display the output image
cv2.imshow("Image", image)
cv2.waitKey(0)